In [1]:
import sys
from timeit import default_timer as timer

sys.path.append('../pmm/')

import inputs
import pmm
import conversions
import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.analysis.rms import rmsd
import numpy as np
#import numpy.linalg 
import nglview as nv

In [2]:
# gather the electronic properties
start = timer()
gauss_file = 'thioindigo/thio_pbe0_esp{}_x180.log'
n_el_states = 16
pmm_inputs = inputs.get_tot_input_gauss(gauss_file, n_el_states)
end = timer()
print('inputs da Gaussian ci mette: ', end - start)
#print(pmm_inputs)

qc_geom = pmm.convert2Universe(pmm_inputs['geometry'])
print(qc_geom)

inputs da Gaussian ci mette:  0.13544240000192076
<Universe with 28 atoms>
/mnt/d/miniconda3/envs/pmm/lib/python3.9/site-packages/MDAnalysis/core/universe.py:467: UserWarning: Residues specified but no atom_resindex given.  All atoms will be placed in first Residue.
  warnings.warn(
/mnt/d/miniconda3/envs/pmm/lib/python3.9/site-packages/MDAnalysis/core/universe.py:473: UserWarning: Segments specified but no segment_resindex given.  All residues will be placed in first Segment
  warnings.warn(


In [3]:
# Read the MD trajectory
start = timer()
traj = mda.Universe('thioindigo/thioindigo_trans_etoh_md.tpr', 'thioindigo/thio_etoh_center_10ps.xtc')
end = timer()
print('leggere xtc ci mette: ', end - start)
w = nv.show_mdanalysis(traj)
w

#qc_traj, solv_traj = pmm.split_qc_solv(traj, '1:28')
#print(qc_traj)
#w = nv.show_mdanalysis(solv_traj)
#w

leggere xtc ci mette:  0.30158070003381


NGLWidget(max_frame=10)

In [29]:
# Let's consider only a single frame
for i, ts in enumerate(traj.trajectory):
    # Divide the trajectory in QC and solvent.
    # NOTE: the indexes for the QC are inclusive of the extremes.
    qc_traj, solv_traj = pmm.split_qc_solv(traj, '1:28')
    #print(pmm.rotate_dip_matrix(pmm_inputs['dip_matrix'], qc_traj, qc_geom))

    # Calculate the center of mass of QC
    cdm = qc_traj.center_of_mass()
    #print(solv_traj.atoms.positions)
    #print('cdm ref ', qc_geom.atoms.center_of_mass())
    # shift origin to the centers of mass of frame_geom to (0, 0, 0).
    # NOTE: it changes the positions in traj_geom permanently.
    qc_traj.atoms.positions = qc_traj.atoms.positions - cdm
    print('cdm traj ', qc_traj.center_of_mass())
    rot_matrix, rmsd = align.rotation_matrix(qc_traj.atoms.positions,
                                             qc_geom.atoms.positions,
                                             weights=qc_geom.atoms.masses)
    #print(rot_matrix)
    if i == 7:
        rot_traj_geom = mda.Universe.empty(qc_geom.atoms.n_atoms, trajectory=True)
        rot_traj_geom.atoms.positions = rot_matrix.dot(qc_traj.atoms.positions.T).T
        print(type(qc_traj), type(qc_geom), type(rot_traj_geom))
        u_tot = mda.Merge(qc_traj, qc_geom.atoms, rot_traj_geom.atoms)
        print(qc_traj.positions)
        print(qc_geom.atoms.positions)
        print(rot_traj_geom.atoms.positions)
        w = nv.show_mdanalysis(u_tot)
w

cdm traj  [-6.72337041e-08 -1.57696418e-08  3.32296128e-09]
cdm traj  [-5.09601909e-08 -1.29904379e-08  5.21633353e-10]
cdm traj  [-6.43249185e-08 -6.61249974e-09 -3.36625662e-11]
cdm traj  [ 3.20704719e-08 -1.97913107e-09 -2.30857997e-09]
cdm traj  [ 3.36811121e-08 -1.42766481e-09  2.93367734e-09]
cdm traj  [ 4.44365762e-08  2.07016724e-08 -5.93400954e-09]
cdm traj  [-5.98251894e-08  7.80078821e-09 -4.95316996e-09]
cdm traj  [-2.25087071e-08  2.34845905e-09  6.06635208e-09]
<class 'MDAnalysis.core.groups.AtomGroup'> <class 'MDAnalysis.core.universe.Universe'> <class 'MDAnalysis.core.universe.Universe'>
[[ 6.134037   -1.754783   -0.7564516 ]
 [ 5.134035   -1.334783   -0.816453  ]
 [ 5.134035   -0.05478228 -1.2564516 ]
 [ 5.954035    0.20521605 -1.9164515 ]
 [ 4.0740356   0.7852179  -1.0364523 ]
 [ 4.054035    1.7652174  -1.4964514 ]
 [ 2.9740336   0.24521697 -0.41645145]
 [ 2.954035   -1.0747827   0.02354717]
 [ 4.0340347  -1.874782   -0.19645214]
 [ 3.994036   -2.954784   -0.01645184]

NGLWidget()